##### EEEL 4220 Final Project
## Wind Farm Sizing and Siting on IEEE-14 System

2. 14-bus single-scenario, no wind planning. Then add transmission line constraints and
introduce the bus angle. You can start by setting the line flow to 9999, which should produce
the same result as the one bus example, then gradually adjust the flow limit.

In [49]:
# Import libraries
import cvxpy as cp
import numpy as np
import pandas as pd

In [50]:
# Load data
b_df = pd.read_csv('buses.csv')
g_df = pd.read_csv('generators.csv')
l_df = pd.read_csv('branches.csv')
d_df = pd.read_csv('demand_scenarios.csv')

In [51]:
# Parameters
c0 = g_df['c0'].to_numpy() # quadratic cost coefficients for generator g
c1 = g_df['c1'].to_numpy() # linear cost coefficients for generator g
c2 = g_df['c2'].to_numpy() # fixed cost coefficients for generator g
Pmin = g_df['Pmin_MW'].to_numpy() # minimum generation limits for generator g
Pmax = g_df['Pmax_MW'].to_numpy() # maximum generation limits for generator g
x = l_df['x_pu'].to_numpy() # reactance of transmission line l
fmax = l_df['rateA_MW'].to_numpy() # thermal limit of line l
D = d_df.loc[d_df['scenario_id']==5,'Pd_MW'].to_numpy() # demand at bus b in scenario s

# Decision variables
p = cp.Variable(len(g_df), nonneg=True) # power output of generator g under scenario s
f  = cp.Variable(len(l_df)) # power flow on line l under scenario s
theta = cp.Variable(len(b_df)) # voltage angle at bus b (radians)

In [52]:
# Initialize incidence matrices
# Generator–bus
# Ggb = 1 if generator g is connected to bus b, 0 otherwise.
G = np.zeros((len(g_df), len(b_df)))

# Line–bus
# Alb = 1 if bus b is the sending (“from”) end of line l, if bus b is the receiving (“to”) end of line l, 0 otherwise.
A = np.zeros((len(l_df), len(b_df)))

# Populate incidence matrices
# Generator–bus
for gen_index in range(len(g_df)):
    bus_id = g_df.loc[gen_index, 'bus_id']
    bus_index = bus_id - 1
    G[gen_index, bus_index] = 1

# Line–bus
for line_index in range(len(l_df)):
    from_bus_id = l_df.loc[line_index, 'from_bus']
    to_bus_id = l_df.loc[line_index, 'to_bus']
    from_bus_index = from_bus_id - 1
    to_bus_index = to_bus_id - 1
    A[line_index, from_bus_index] = 1
    A[line_index, to_bus_index] = -1

In [53]:
# Define constraints
# Initialize an empty constraint set
con = [] 

# power balance
con.append((A.T @ f) + D == (G.T @ p))

# generator output limits
con.append(p <= Pmax)  # maximum generation

# set reference angle
con.append(theta[0] == 0)

# DC power flow
con.append(f == (A @ theta) / x)

# line flow limits
con.append(f <= fmax)  # maximum line flow
con.append(f >= -fmax)  # minimum line flow

In [54]:
# Define objective function - total cost
obj = cp.Minimize(cp.sum(cp.multiply(c2, cp.square(p)) + cp.multiply(c1, p) + c0))

In [55]:
# Solve the problem
prob1 = cp.Problem(obj, con)
prob1.solve(solver = "HIGHS");

In [56]:
# results
print("\n The total generation in the system is:")
print((p.value.sum().round(1)))
print("The total demand in the system is:") 
print(D.sum(axis=0).round(1))

print("\n Expected operating cost:")
print(obj.value.round(1))

print("\n Thermal generator dispatch results: ")
GT = pd.DataFrame(p.value, index = ['G1', 'G2', 'G3', 'G4', 'G5'])
print(GT.round(1))

print("\n Line flow results: ")
LT = pd.DataFrame(f.value, index = ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20'])
print(LT.round(1))

print("\n Locational Marginal Prices: ")
MCT = pd.DataFrame(con[0].dual_value, index = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'B12', 'B13', 'B14'])
print(MCT.round(1))


 The total generation in the system is:
272.0
The total demand in the system is:
272.0

 Expected operating cost:
899.2

 Thermal generator dispatch results: 
       0
G1  93.1
G2  49.9
G3  51.9
G4  41.2
G5  35.9

 Line flow results: 
        0
F1   66.1
F2   27.0
F3   36.3
F4   27.6
F5   29.2
F6  -10.7
F7  -27.2
F8   -8.4
F9    2.4
F10  21.0
F11  16.4
F12  10.5
F13  23.6
F14 -35.9
F15  27.5
F16  -3.2
F17   2.1
F18 -12.7
F19   4.1
F20  13.5

 Locational Marginal Prices: 
       0
B1   4.8
B2   4.8
B3   4.8
B4   4.8
B5   4.8
B6   4.8
B7   4.8
B8   4.8
B9   4.8
B10  4.8
B11  4.8
B12  4.8
B13  4.8
B14  4.8
